In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


In [2]:

img_height = 150
img_width = 150
batch_size = 32
data_directory = '/Users/baudi/AI/practicas/uvas/data/train_val/'
test_data_directory = '/Users/baudi/AI/practicas/uvas/data/test/'
val_split = 0.2
seed = 42
num_classes = 4
learning_rate = 0.0001


In [3]:

def adjustments(image):
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.9, 1.1)
    return image

def process_labels(images, labels):
    one_hot_labels = tf.one_hot(labels, depth=num_classes)
    return images, one_hot_labels

train_data_raw = image_dataset_from_directory(
    data_directory,
    validation_split=val_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_data_raw.class_names

train_data = train_data_raw.map(lambda x, y: (adjustments(x), y)).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)


val_data = image_dataset_from_directory(
    data_directory,
    validation_split=val_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)

test_data = image_dataset_from_directory(
    test_data_directory,
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)


Found 3248 files belonging to 4 classes.
Using 2599 files for training.
Metal device set to: Apple M2 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

Found 3248 files belonging to 4 classes.
Using 649 files for validation.
Found 814 files belonging to 4 classes.


2023-04-10 12:05:07.958054: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-10 12:05:07.958224: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
def build_model():

    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(512, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(4, activation='softmax')  # 4 clases: sanas, enfermedad1, enfermedad2, enfermedad3
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    return model


In [5]:

def cross_validate(model_builder, n_splits=2, epochs=20):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    X = np.concatenate([x for x, y in train_data] + [x for x, y in val_data])
    y = np.concatenate([y for x, y in train_data] + [y for x, y in val_data])

    accuracies = []
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y), start=1):
        model = model_builder()
        print(f"Comienza el entrenamiento del fold {fold}")
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001)

        callbacks = [early_stopping, reduce_lr]

        history = model.fit(X[train_idx],
                  y[train_idx],
                  validation_data=(X[val_idx], y[val_idx]),
                  batch_size = batch_size,
                  epochs=epochs,
                  callbacks=callbacks)
        print(f"Termina el entrenamiento del fold {fold}")
        val_accuracy = history.history['val_accuracy']
        last_val_accuracy = val_accuracy[-1]
        print(f"Precisión de validación del fold {fold}: {last_val_accuracy}")

        accuracies.append(last_val_accuracy)

    return np.mean(accuracies)


In [6]:

mean_accuracy = cross_validate(build_model,5,20)
print(f'Mean accuracy: {mean_accuracy:.4f}')


2023-04-10 12:05:08.243021: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Comienza el entrenamiento del fold 1
Epoch 1/20


2023-04-10 12:05:09.395396: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


82/82 [==============================] - ETA: 0s - loss: 0.8736 - accuracy: 0.7267

2023-04-10 12:05:13.742413: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

82/82 [==============================] - 5s 55ms/step - loss: 0.8956 - accuracy: 0.7195 - val_loss: 1.0455 - val_accuracy: 0.6225 - lr: 1.0000e-04
Epoch 2/20
82/82 [==============================] - 4s 47ms/step - loss: 0.3130 - accuracy: 0.8934 - val_loss: 0.6842 - val_accuracy: 0.7381 - lr: 1.0000e-04
Epoch 3/20
82/82 [==============================] - 4s 48ms/step - loss: 0.2003 - accuracy: 0.9319 - val_loss: 0.3900 - val_accuracy: 0.8521 - lr: 1.0000e-04
Epoch 4/20
82/82 [==============================] - 4s 47ms/step - loss: 0.1201 - accuracy: 0.9577 - val_loss: 0.3401 - val_accuracy: 0.8644 - lr: 1.0000e-04
Epoch 5/20
82/82 [==============================] - 4s 47ms/step - loss: 0.0882 - accuracy: 0.9704 - val_loss: 0.2276 - val_accuracy: 0.9168 - lr: 1.0000e-04
Epoch 6/20
82/82 [==============================] - 4s 47ms/step - loss: 0.0725 - accuracy: 0.9750 - val_loss: 0.2768 - val_accuracy: 0.9014 - lr: 1.0000e-04
Epoch 7/20
82/82 [==============================] - 4s 47ms/ste

2023-04-10 12:10:22.429231: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


82/82 [==============================] - ETA: 0s - loss: 0.8705 - accuracy: 0.7318

2023-04-10 12:10:26.679466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


82/82 [==============================] - 5s 54ms/step - loss: 0.8705 - accuracy: 0.7318 - val_loss: 1.1746 - val_accuracy: 0.5763 - lr: 1.0000e-04
Epoch 2/20
82/82 [==============================] - 4s 47ms/step - loss: 0.3001 - accuracy: 0.8927 - val_loss: 0.8123 - val_accuracy: 0.6810 - lr: 1.0000e-04
Epoch 3/20
82/82 [==============================] - 4s 47ms/step - loss: 0.1768 - accuracy: 0.9396 - val_loss: 0.3807 - val_accuracy: 0.8490 - lr: 1.0000e-04
Epoch 4/20
82/82 [==============================] - 4s 47ms/step - loss: 0.1085 - accuracy: 0.9631 - val_loss: 0.1897 - val_accuracy: 0.9337 - lr: 1.0000e-04
Epoch 5/20
82/82 [==============================] - 4s 47ms/step - loss: 0.0742 - accuracy: 0.9738 - val_loss: 0.1736 - val_accuracy: 0.9353 - lr: 1.0000e-04
Epoch 6/20
82/82 [==============================] - 4s 47ms/step - loss: 0.0739 - accuracy: 0.9742 - val_loss: 0.2353 - val_accuracy: 0.9076 - lr: 1.0000e-04
Epoch 7/20
82/82 [==============================] - 4s 47ms/ste

In [7]:

def train_final_model(model_builder, epochs=20):
    X = np.concatenate([x for x, y in train_data] + [x for x, y in val_data])
    y = np.concatenate([y for x, y in train_data] + [y for x, y in val_data])

    final_model = model_builder()

    # Definir el callback de EarlyStopping
    early_stopping = EarlyStopping(monitor='loss', patience=3, min_delta=0.001)

    # Agregar el callback al método fit
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001)

    callbacks = [early_stopping, reduce_lr]
    final_model.fit(X, y, epochs=epochs, batch_size=batch_size, callbacks=callbacks)

    return final_model

final_model = train_final_model(build_model,50)


Epoch 1/50


2023-04-10 12:11:41.176084: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


102/102 [==============================] - 6s 47ms/step - loss: 0.7858 - accuracy: 0.7445 - lr: 1.0000e-04
Epoch 2/50
102/102 [==============================] - 5s 45ms/step - loss: 0.2857 - accuracy: 0.9018 - lr: 1.0000e-04
Epoch 3/50
102/102 [==============================] - 4s 43ms/step - loss: 0.1870 - accuracy: 0.9338 - lr: 1.0000e-04
Epoch 4/50
102/102 [==============================] - 4s 43ms/step - loss: 0.1127 - accuracy: 0.9627 - lr: 1.0000e-04
Epoch 5/50
102/102 [==============================] - 4s 43ms/step - loss: 0.0756 - accuracy: 0.9729 - lr: 1.0000e-04
Epoch 6/50
102/102 [==============================] - 4s 44ms/step - loss: 0.0609 - accuracy: 0.9757 - lr: 1.0000e-04
Epoch 7/50
102/102 [==============================] - 4s 43ms/step - loss: 0.0510 - accuracy: 0.9794 - lr: 1.0000e-04
Epoch 8/50
102/102 [==============================] - 4s 43ms/step - loss: 0.0317 - accuracy: 0.9898 - lr: 1.0000e-04
Epoch 9/50
102/102 [==============================] - 4s 43ms/step 

In [8]:
test_loss, test_accuracy = final_model.evaluate(test_data)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Y_pred = final_model.predict(test_data)
y_pred = np.argmax(Y_pred, axis=1)

y_true = np.concatenate([y.numpy() for _, y in test_data.unbatch()])
y_true_labels = np.argmax(y_true.reshape(-1, len(class_names)), axis=1)

print('Classification Report:')
print(classification_report(y_true_labels, y_pred, target_names=class_names))

print('Confusion Matrix:')
print(confusion_matrix(y_true_labels, y_pred))


 4/26 [===>..........................] - ETA: 0s - loss: 0.0354 - accuracy: 0.9844

2023-04-10 12:15:24.406105: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


26/26 [==============================] - 1s 18ms/step - loss: 0.0864 - accuracy: 0.9767
Test Loss: 0.0864
Test Accuracy: 0.9767
 6/26 [=====>........................] - ETA: 0s

2023-04-10 12:15:25.047720: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


26/26 [==============================] - 0s 14ms/step
Classification Report:
                                            precision    recall  f1-score   support

                         Grape___Black_rot       0.99      0.94      0.97       236
              Grape___Esca_(Black_Measles)       0.96      0.99      0.98       277
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)       0.99      0.99      0.99       216
                           Grape___healthy       0.97      0.99      0.98        85

                                  accuracy                           0.98       814
                                 macro avg       0.98      0.98      0.98       814
                              weighted avg       0.98      0.98      0.98       814

Confusion Matrix:
[[223  10   2   1]
 [  1 275   0   1]
 [  2   0 213   1]
 [  0   0   1  84]]
